# Solving the 6 Player Intransitive Dice Problem 

After I had published my 4- and 5-player problems, I was contacted by another recreational mathematician, Youhua Li, who had a solution for the four-player problem that involved fewer dice.  Based on some initial analysis of his solution, I conjectured that this set of dice could be generated by and satisfy an arbitrary domination tournament graph, and this "publication" intends to prove that.  

In addition, another recreational mathematician John Robert Marshall commented on the math_problems repo that he had beaten me to the punch with a published solution for four-player dice. These are notably is easy to extend to five- and six-player solutions. As he noted (in a private communication) that an isomorphic set to the 71 dice in my five-player solution was verified already, we should check those as well for the six-player variant in particular.

First, some imports and functions from the last one:

In [1]:
# First let's import
import itertools as it
from tqdm import tqdm
from joblib import delayed,Parallel
from tabulate import tabulate
import pandas as pd

We also define some previous functions that are involved in the checking.  In particular, we modify the checking function to check only values that include the minimum value and are above it.  So for instance, the subcombinations for n=7 that include 1 are:

In [2]:
def get_players(n): return sum([n>=i for i in [1, 3, 7, 19, 67, 331, 1163]])
#  def get_sub_comb(n,minimum): return ((0,minimum,*j) for j in it.combinations([i for i in range(n) if i>minimum], get_players(n)-3))
def get_sub_comb(n,minimum): return ((minimum,*j) for j in it.combinations([i for i in range(n) if i>minimum], get_players(n)-2))
def get_win_bias(d1,d2): return sum(((j<i)-(i<j) for i,j in zip(d1,d2)))
def get_winners(dice): return tuple(tuple(d1[0] for d1 in dice if get_win_bias(d1,d2)>0) for d2 in dice)
def has_full_coverage(winners,n,minimum):
    samples = get_sub_comb(n,minimum)
    set_winners = list(map(set,winners))
    return all(any(all(i in w for i in s) for w in set_winners) for s in tqdm(samples,position=0))

list(get_sub_comb(n=7,minimum=0))

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6)]

In addition, we define the functions to generate the dice.  Note that the graph functions for either set are slightly different:

In [3]:
def li_dice(n):
    assert not (n+1)%4
    w = tuple(sorted([((i+1)**2)%n for i in range(n//2)]))
    return list(zip(*sorted([[(i+j*k)%n+i*n*len(w)+w.index(j)*n for k in range(n)] for i in range(n) for j in w])))
def marshall_dice(n):
    assert not (n+1)%8
    g = tuple([(2**i)%n for i in range(n//2)])
    return [tuple([(i*j+n//2)%n+k*n for k,i in enumerate(g)]) for j in range(-n//2+1,n//2+1)]

Let's quickly check our coverage testing function. Let's give it a positive result, testing for the tournament that represents a solution for the Oskar (n=7) dice:

In [4]:
n = 7
dice = li_dice(n)
winners = get_winners(dice)
print(winners)
print({i:has_full_coverage(winners,n,i) for i in range(0,n-get_players(n)+2)})

((1, 2, 4), (2, 3, 5), (3, 4, 6), (0, 4, 5), (1, 5, 6), (0, 2, 6), (0, 1, 3))


6it [00:00, 118706.72it/s]
5it [00:00, 65741.44it/s]
4it [00:00, 70789.94it/s]
3it [00:00, 63872.65it/s]
2it [00:00, 44858.87it/s]
1it [00:00, 25731.93it/s]

{0: True, 1: True, 2: True, 3: True, 4: True, 5: True}


Now let's modify the tournament such that it no longer works, removing the last value from the set.  As a result, (0,3) and (1,3) no longer have any items in the "winners" list that contain both those values, and as a result, 0 and 1 should fail but the others should remain unaffected:

In [5]:
modified_winners = tuple(list(winners[:-1])+[winners[-1][:-1]])
print(modified_winners)
print({i:has_full_coverage(modified_winners,n,i) for i in range(0,n-get_players(n)+2)})

((1, 2, 4), (2, 3, 5), (3, 4, 6), (0, 4, 5), (1, 5, 6), (0, 2, 6), (0, 1))


2it [00:00, 32388.45it/s]
1it [00:00, 6009.03it/s]
4it [00:00, 51463.85it/s]
3it [00:00, 62601.55it/s]
2it [00:00, 31418.01it/s]
1it [00:00, 24966.10it/s]

{0: False, 1: False, 2: True, 3: True, 4: True, 5: True}


So we have confidence that our coverage function is working as expected.  What we need to do is parallelize the creation of the winners list. Also, we need a function that double checks that the dominating set in our winners list is one single unique value.  So, for instance, for the n=7 dice, 0 beats {1,2,4}; 1 beats {2,3,5}; etc. each die beating the same counts above and below.  Then we can leverage our get_sub_comb function to simply look at the dice that contain a zero as the minimum value, which will result in us proving that any value works by rotational symmetry. (Technically we could further optimize this by looking at intervals and doing a cumulative sum, such that the interval is largest before the 0, but this will complicate our analysis and simplification is preferred even if it results in some degenerate values being checked).

Thus if the tournament has a unique dominating set AND the solutions that contain a minimum value of zero are fully dominated then we know that any arbitrary set of dice with a minimum die of m can be dominated by m plus the index of the domination that occurs when m=0, therefore satisfying the condition that the dice are fully dominated.

In [6]:
def check(d2,dice): return tuple([d1[0] for d1 in dice if get_win_bias(d1,d2)>0])
def get_winners_parallel(dice): return tuple(Parallel(n_jobs=-1,verbose=2)(delayed(check)(d2,dice) for d2 in dice))

def has_unique_dominating_set(winners,n): return 1==len({tuple(sorted([(j-k)%n for j in i])) for k,i in enumerate(winners)})
def check_if_non_transitive(dice):
    n = len(dice)
    winners = get_winners_parallel(dice)
    return has_full_coverage(winners,n,0) and has_unique_dominating_set(winners,n)

And the Li and Marshall dice for a 5-player solutions work:

In [7]:
check_if_non_transitive(li_dice(67))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 out of  67 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  67 out of  67 | elapsed:    0.5s finished
45760it [00:00, 261151.70it/s]


True

In [8]:
check_if_non_transitive(li_dice(71))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 out of  71 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  71 out of  71 | elapsed:    0.2s finished
54740it [00:00, 205266.94it/s]


True

It should be noted that once the winner table is generated, whichever has the smallest number of dice will simply run faster.

Now for checking the six-player solution, the Marshall dice have the restriction that n = 7 (mod 8) - the reasoning is given in the four-player solution in this repo - of which the possible values for n>=331 required for a 6-player are given:

In [9]:
[i for i in range(7,400,8) if i>=331 and __import__('sympy').isprime(i)]

[359, 367, 383]

Let's check the Li dice first though:

In [10]:
check_if_non_transitive(li_dice(331))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-1)]: Done 118 tasks      | elapsed:   57.9s
[Parallel(n_jobs=-1)]: Done 331 out of 331 | elapsed:  2.7min finished
485199330it [1:47:11, 75445.86it/s] 


True

So there we have it.  The Dice given by Li represent a working solution to the six-player intransitive dice problem.  We can also attempt this with the Marshall dice form for 6 players:

In [11]:
check_if_non_transitive(marshall_dice(359))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-1)]: Done 252 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 359 out of 359 | elapsed:    0.4s finished
673005095it [1:54:38, 97845.60it/s] 


True

We can officially call the six-player problem solved!

- The smallest set of dice are given by Li, which satisfies the tournament with the dominating directional graph of size 331
- The fewest faces are given by Marshall, which satisfies the tournament with a set of 359 dice.

And if one is interested, the CSV files that contain the respective dice can be viewed in the repo and are generated here:

In [12]:
pd.DataFrame(li_dice(331)).to_csv('Li_331.csv')
df = pd.DataFrame(marshall_dice(359))
df.to_csv('Marshall_359.csv')
print(df)

     0    1    2     3     4     5     6     7     8     9    ...    169  \
0      0  539  899  1260  1623  1990  2365  2756  3179  3307  ...  60745   
1      1  541  903  1268  1639  2022  2429  2525  3076  3460  ...  60894   
2      2  543  907  1276  1655  2054  2493  2653  2973  3254  ...  60684   
3      3  545  911  1284  1671  2086  2198  2781  3229  3407  ...  60833   
4      4  547  915  1292  1687  2118  2262  2550  3126  3560  ...  60982   
..   ...  ...  ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
354  354  529  879  1220  1543  1830  2404  2834  2976  3260  ...  60718   
355  355  531  883  1228  1559  1862  2468  2603  2873  3413  ...  60867   
356  356  533  887  1236  1575  1894  2173  2731  3129  3566  ...  61016   
357  357  535  891  1244  1591  1926  2237  2859  3026  3360  ...  60806   
358  358  537  895  1252  1607  1958  2301  2628  2923  3513  ...  60955   

       170    171    172    173    174    175    176    177    178  
0    61358  61507 